# 报表设计学习目标
- 知道报表的作用
- 知道搭建日报的基本流程
- 应用SQL、Excel实现日报开发


--- 

# 1、报表开发的要点？
- 数据整合
- 指标体系建设
- 可视化（模版化-仪表板）

# 2、报表的作用是什么？
概念：将海量的业务数据有效的组织展示出来，将有价值的信息（事物的现状、问题、原因、本质和规律）传递出去，辅助公司业务/战略部门作出决策

# 3、日报搭建的基本流程有哪些？

![日报搭建](./img/日报搭建.png)

- 搭建核心指标体系
- 多源报表的汇总整理
- 自动化数据计算
- 报表模版设计


# 4、指标搭建五步走
- 第一步：搭建指标体系，先明确产品类型，明确业务究竟是什么，目标是什么。梳理业务流程，形成一个指标体系框架。不同的业务类型的产品会有不同的指标体系框架。
- 第二步：搭建完指标体系后，需要对指标体系进行拆分，确定我们需要的核心指标，然后从纬度和度量角度去计算
- 第三步：指标体系方案敲定后，开始整理底层数据存储逻辑，明确每个字段要从哪里获取，每张表应该涵盖哪些指标哪些字段纬度，或者遇到问题
    - 缺少必要数据（没有埋点）
    - 有埋点但是数据未合理入库
    - 数据缺失、错误严重
- 第四步：指标体系可视化展示
- 第五步：存档、追踪、落地、优化



In [12]:
%%sql
-- 核心指标计算
select date                    as 日期,
       sum(liulan)             as 页面浏览次数,
       sum(buyer)              as 卖家访问次数,
       sum(on_sell)            as 平均在售商品数,
       sum(refund_ratio) / 100 as 退款率,
       sum(sales_total)        as 销量,
       sum(gmv_total)          as 销售额
from ds_sales_all
group by date
order by date desc
limit 7;

,日期,页面浏览次数,卖家访问次数,平均在售商品数,退款率,销量,销售额
0,2020-12-31,1871,1259,88,0.027121,127,4747.498444
1,2020-12-30,2763,1722,87,0.061816,123,3683.890844
2,2020-12-29,2692,1744,88,0.018265,162,5736.666250
3,2020-12-28,2720,1858,92,0.018978,153,5593.030658
4,2020-12-27,2710,1822,93,0.041386,143,5111.967898
5,2020-12-26,3485,2260,92,0.025157,178,6434.156536
6,2020-12-25,3457,2222,91,0.001944,214,8429.858188


In [13]:
%%sql
# 分品牌统计
select update_day       as 日期,
       brand            as 品牌,
       sum(units_count) as 销量,
       sum(gmv)         as 销售额
from ds_sales_detail
where update_day = (select max(update_day) from ds_sales_detail)
group by update_day, brand;

,日期,品牌,销量,销售额
0,2020-12-31,AA,16,844.84
1,2020-12-31,BB,11,310.09
2,2020-12-31,CC,54,2016.96
3,2020-12-31,EE,15,419.85
4,2020-12-31,DD,10,595.56


In [14]:
%%sql
# 分链接统计
# cost/clicks 计算了点一下花了多少钱
# clicks/impressions as ctr 点击/曝光计算了点击率
select end_day,
       country,
       brand,
       asin,
       clicks,
       cost,
       impressions,
       gmv_ad,
       sales_ad,
       cost / clicks        as cpc,
       clicks / impressions as ctr
from ds_sales_ad;

,end_day,country,brand,asin,clicks,cost,impressions,gmv_ad,sales_ad,cpc,ctr
0,2020-12-01,Ca,AA,B072LW84XX,29,16.12,4393,0.00,0,0.555862,0.0066
1,2020-12-01,Ca,AA,B074M6KGXX,7,5.94,1842,0.00,0,0.848571,0.0038
2,2020-12-01,Ca,AA,B074M766XX,34,24.43,6105,69.88,2,0.718529,0.0056
3,2020-12-01,Ca,BB,B072BP12XX,30,9.08,8372,0.00,0,0.302667,0.0036
4,2020-12-01,Ca,CC,B078NT86XX,27,12.59,9167,56.21,2,0.466296,0.0029
...,...,...,...,...,...,...,...,...,...,...,...
1518,2020-12-31,Us,EE,B07QY6R8XX,3,2.21,3308,0.00,0,0.736667,0.0009
1519,2020-12-31,Us,EE,B07R1HZVXX,8,5.56,4207,0.00,0,0.695000,0.0019
1520,2020-12-31,Us,EE,B07R2KP9XX,3,1.62,1052,0.00,0,0.540000,0.0029
1521,2020-12-31,Us,EE,B07S4HN2XX,0,0.00,38,0.00,0,NaN,0.0000


In [15]:
%%sql
# 网页信息
select country,
       insert_date,
       asin,
       sales_rank rank_big,
       rank_small,
       stars,
       price,
       reviews,
       daily_units
from ds_sales_url;

,country,insert_date,asin,rank_big,rank_small,stars,price,reviews,daily_units
0,Us,2020-12-01,B01M29YGXX,4668,142,4.4,42.99,134,5
1,Us,2020-12-01,B077ZRP3XX,217,10,4.3,29.99,369,40
2,Us,2020-12-01,B07QBPSQXX,31251,49,4.9,43.99,23,2
3,Us,2020-12-01,B07QBPTNXX,27081,43,4.9,41.99,23,3
4,Us,2020-12-01,B07QLPG6XX,189002,222,4.5,45.99,25,1
...,...,...,...,...,...,...,...,...,...
3796,Us,2020-12-31,B07HNTJ5XX,25388,373,4.5,48.99,133,0
3797,Us,2020-12-31,B07HNXVZXX,20808,145,4.6,43.99,164,0
3798,Us,2020-12-31,B07HNXPXXX,10570,84,4.5,47.99,133,1
3799,Us,2020-12-31,B01N4876XX,27004,413,4.6,46.99,164,0


In [16]:
%%sql

# 销售详情
select detail.update_day,
       detail.country,
       detail.brand,
       detail.asin,
       detail.units_count,
       detail.gmv,
       t_sales.`最晚售出距今天数`,
       t_sales.`近7日销量`,
       t_sales.`近7日平均销量`,
       t_sales.`日均销量预估`,
       detail.profit
from ds_sales_detail detail
    left join(
        select
            brand,
            country,
            asin,
            datediff('2020-12-31', max(update_day))                                                                             as '最晚售出距今天数',
            sum(case when DATEDIFF('2020-12-31', update_day) <= 7 then units_count else 0 end)                                  as '近7日销量',
            round(sum(case when DATEDIFF('2020-12-31', update_day) <= 7 then units_count else 0 end) / 7,2)                     as '近7日平均销量',
            round((
                round(sum(case when DATEDIFF('2020-12-31', update_day) <= 7  then units_count else 0 end) /  7, 2) * 0.6 +
                round(sum(case when DATEDIFF('2020-12-31', update_day) <= 15 then units_count else 0 end) / 15, 2) * 0.3 +
                round(sum(case when DATEDIFF('2020-12-31', update_day) <= 30 then units_count  else 0 end) / 30, 2) * 0.1
            ), 2)                                                                                                                as '日均销量预估'
        from ds_sales_detail
        where units_count != 0
        group by brand, country, asin
    ) t_sales on detail.asin = t_sales.asin and detail.country = t_sales.country and detail.brand = t_sales.brand
where detail.update_day = '2020-12-31'
order by country, brand, units_count desc;

,update_day,country,brand,asin,units_count,gmv,最晚售出距今天数,近7日销量,近7日平均销量,日均销量预估,profit
0,2020-12-31,Ca,AA,B074M766XX,4,141.57,0,24,3.43,3.83,15.11
1,2020-12-31,Ca,AA,B074M6KGXX,0,0.00,1,26,3.71,6.34,27.08
2,2020-12-31,Ca,BB,B0748DQ4XX,0,0.00,1,5,0.71,0.59,17.54
3,2020-12-31,Ca,CC,B078NT86XX,0,0.00,2,13,1.86,2.55,1.05
4,2020-12-31,Ca,DD,B06VTR57XX,2,84.64,0,34,4.86,4.67,21.76
5,2020-12-31,Jp,AA,B07XCMZ1XX,1,28.38,0,12,1.71,1.88,23.36
6,2020-12-31,Jp,AA,B07X9LFKXX,0,0.00,2,5,0.71,0.79,1.31
7,2020-12-31,Jp,BB,B07R9XMRXX,0,0.00,1,8,1.14,1.55,1.40
8,2020-12-31,Jp,BB,B07R7PRDXX,0,0.00,5,3,0.43,0.75,1.16
9,2020-12-31,Jp,CC,B07VNCB3XX,0,0.00,4,6,0.86,2.03,2.79


In [17]:
%%sql

# 合并起来
select distinct detail.update_day                                日期,
                detail.country                                   国家,
                detail.brand                                     品牌,
                detail.asin                                      链接ASIN,
                detail.units_count + 0                           销量,
                detail.gmv + 0                                   销售额,
                detail.`最晚售出距今天数` + 0                    最晚售出距今天数,
                detail.`近7日销量` + 0                           近7日销量,
                detail.`近7日平均销量` + 0                       近7日平均销量,
                detail.`日均销量预估` + 0                        日均销量预估,
                stock.in_stock + 0                               当前库存,
                round(stock.in_stock / detail.`日均销量预估`, 0) 售完预计天数,
                ad.clicks + 0                                    点击数,
                ad.cost + 0                                      广告花费,
                ad.impressions + 0                               曝光量,
                ad.gmv_ad + 0                                    广告带来销售额,
                ad.sales_ad + 0                                  广告带来销量,
                ad.cpc + 0                                       按点击收费CPC,
                ad.ctr + 0                                       转化率CTR,
                url.rank_big + 0                                 大类排名,
                url.rank_small + 0                               小类排名,
                url.reviews + 0                                  评论数,
                url.price + 0                                    价格,
                url.stars + 0                                    星级
from (
    select
        detail1.update_day,
        detail1.country,
        detail1.brand,
        detail1.asin,
        detail1.units_count,
        detail1.gmv,
        t_sales.`最晚售出距今天数`,
        t_sales.`近7日销量`,
        t_sales.`近7日平均销量`,
        t_sales.`日均销量预估`,
        detail1.profit
    from ds_sales_detail detail1
        left join(
            select
                brand,
                country,
                asin,
                datediff('2020-12-31', max(update_day))                                                                         as '最晚售出距今天数',
                sum(case when DATEDIFF('2020-12-31', update_day) <= 7 then units_count else 0 end)                              as '近7日销量',
                round(sum(case when DATEDIFF('2020-12-31', update_day) <= 7 then units_count else 0 end) / 7, 2)                as '近7日平均销量',
                # 加权平均预测
                round((
                    round(sum(case when DATEDIFF('2020-12-31', update_day) <= 7  then units_count else 0 end) /  7, 2) * 0.6 +
                    round(sum(case when DATEDIFF('2020-12-31', update_day) <= 15 then units_count else 0 end) / 15, 2) * 0.3 +
                    round(sum(case when DATEDIFF('2020-12-31', update_day) <= 30 then units_count else 0 end) / 30, 2) * 0.1
                    ), 2)                                                                                                       as '日均销量预估'
            from ds_sales_detail
            where units_count != 0
            group by brand, country, asin
        ) t_sales on detail1.asin = t_sales.asin and detail1.country = t_sales.country and detail1.brand = t_sales.brand
    where detail1.update_day = '2020-12-31') detail
        left join ds_sales_stock stock on detail.update_day = stock.update_day and detail.asin = stock.asin and detail.country = stock.country and detail.brand = stock.brand
        left join (
            select
                end_day,
                country,
                brand,
                asin,
                clicks,
                cost,
                impressions,
                gmv_ad,
                sales_ad,
                cost / clicks        as cpc,
                clicks / impressions as ctr
            from ds_sales_ad
            ) ad on ad.end_day = detail.update_day and detail.asin = ad.asin and detail.country = ad.country and detail.brand = ad.brand
        left join (
            select
                country,
                insert_date,
                asin,
                sales_rank rank_big,
                rank_small,
                stars,
                price,
                reviews,
                daily_units
            from ds_sales_url
            ) url on detail.asin = url.asin and detail.update_day = url.insert_date and detail.country = url.country
where detail.update_day = (
    select max(update_day)
    from ds_sales_detail
    );


,日期,国家,品牌,链接ASIN,销量,销售额,最晚售出距今天数,近7日销量,近7日平均销量,日均销量预估,...,曝光量,广告带来销售额,广告带来销量,按点击收费CPC,转化率CTR,大类排名,小类排名,评论数,价格,星级
0,2020-12-31,Us,AA,B07WJYSSXX,9,494.91,0,56,8.00,13.99,...,5027.0,109.98,2.0,0.790357,0.0056,3842,8,66,54.99,3.9
1,2020-12-31,Ca,AA,B074M766XX,4,141.57,0,24,3.43,3.83,...,702.0,70.78,2.0,0.660000,0.0057,1584,34,135,45.99,4.2
2,2020-12-31,Us,AA,B07PPGQZXX,1,89.99,0,1,0.14,0.32,...,4342.0,89.99,1.0,1.067273,0.0025,61863,868,46,89.99,4.0
3,2020-12-31,Us,AA,B07PN9S8XX,1,89.99,0,4,0.57,1.31,...,NaN,NaN,NaN,NaN,NaN,11063,146,46,89.99,4.0
4,2020-12-31,Ca,AA,B074M6KGXX,0,0.00,1,26,3.71,6.34,...,NaN,NaN,NaN,NaN,NaN,226,3,135,39.99,4.2
5,2020-12-31,Us,BB,B06ZZRQKXX,6,173.94,0,21,3.00,4.90,...,NaN,NaN,NaN,NaN,NaN,7841,97,138,28.99,4.0
6,2020-12-31,Us,BB,B075M1F4XX,3,122.97,0,13,1.86,3.11,...,NaN,NaN,NaN,NaN,NaN,18229,255,147,40.99,4.4
7,2020-12-31,Us,BB,B07NRMHVXX,2,13.18,0,39,5.57,6.39,...,4425.0,13.18,2.0,0.593333,0.0027,4167,21,33,6.59,4.7
8,2020-12-31,Us,BB,B07JXYLXXX,0,0.00,2,9,1.29,2.50,...,4562.0,0.00,0.0,1.132083,0.0053,13988,178,25,79.99,4.2
9,2020-12-31,Ca,BB,B0748DQ4XX,0,0.00,1,5,0.71,0.59,...,19651.0,0.00,0.0,0.325833,0.0031,920,2,122,79.99,3.7
